In [1]:
from pathlib import Path

import pandas as pd
import xarray as xr
from dask.distributed import Client
from seapopym.configuration.no_transport import ForcingParameter, ForcingUnit
from seapopym.standard.units import StandardUnitsLabels

from seapopym_optimization.cost_function import DayCycle, SimpleRootMeanSquareErrorCostFunction, TimeSeriesObservation
from seapopym_optimization.functional_group import NoTransportFunctionalGroup, Parameter
from seapopym_optimization.genetic_algorithm import SimpleGeneticAlgorithm, SimpleGeneticAlgorithmParameters
from seapopym_optimization.model_generator import NoTransportModelGenerator


In [2]:
path_to_forcing = "../../../1_data_processing/1_1_Forcing/data/1_products/Hot_cmems_climato.zarr"
path_to_npp = "../../../1_data_processing/1_1_Forcing/data/1_products/Hot_observed_npp_climato.zarr"
path_to_obs = "../../../1_data_processing/1_1_Forcing/data/1_products/Hot_obs_zoo_climato_monthly_2002_2015.zarr"
export_file_name = "SeapoPym_biomass_HOT_climato_obs_npp_2_groups"

In [3]:
LATITUDE = 22.75
LONGITUDE = -158
TIME_START = "2005-01-02"
TIME_END = "2008-01-01"
STABILIZATION_TIME = 12
SAVE = True

## Loading


### Forcing


In [4]:
forcing = xr.open_zarr(path_to_forcing)
forcing = forcing.sel(time=slice(TIME_START, TIME_END))
forcing["T"].attrs["units"] = StandardUnitsLabels.temperature.units
forcing.load()

<xarray.Dataset> Size: 193kB
Dimensions:              (time: 1095, latitude: 1, longitude: 1, depth: 3)
Coordinates:
  * depth                (depth) int64 24B 0 1 2
  * latitude             (latitude) float64 8B 22.75
  * longitude            (longitude) int64 8B -158
  * time                 (time) datetime64[ns] 9kB 2005-01-02 ... 2008-01-01
Data variables: (12/13)
    T                    (time, latitude, longitude, depth) float64 26kB 23.8...
    U                    (time, latitude, longitude, depth) float64 26kB -0.0...
    V                    (time, latitude, longitude, depth) float64 26kB -0.0...
    mnkc_epi             (time, latitude, longitude) float64 9kB 0.2862 ... 0...
    mnkc_hmlmeso         (time, latitude, longitude) float64 9kB 0.8062 ... 0...
    mnkc_lmeso           (time, latitude, longitude) float64 9kB 2.113 ... 2.116
    ...                   ...
    mnkc_mumeso          (time, latitude, longitude) float64 9kB 0.5676 ... 0...
    mnkc_umeso           (time, latitude, longitude) float64 9kB 0.4922 ... 0...
    npp                  (time, latitude, longitude) float64 9kB 322.7 ... 302.2
    pelagic_layer_depth  (time, latitude, longitude, depth) float64 26kB 113....
    zeu                  (time, latitude, longitude) float64 9kB 75.51 ... 76.5
    zooc                 (time, latitude, longitude) float64 9kB 0.267 ... 0.261
Attributes:
    Conventions:  CF-1.7
    history:      Created on 2024-10-17
    institution:  CLS
    references:   http://www.cls.fr; http://www.seapodym.eu
    source:       SEAPODYM-LMTL 3.0.0
    title:        Global ocean low and mid trophic levels biomass content hin...

### Epipelagic layer


In [5]:
epi_layer_depth = forcing["pelagic_layer_depth"].sel(depth=0).load()
epi_layer_depth = epi_layer_depth.resample(time="1D").mean()
epi_layer_depth.attrs["units"] = "meter"
epi_layer_depth = epi_layer_depth.pint.quantify()
epi_layer_depth

Magnitude,[[[113.25850445781492]] [[113.67641789059907]] [[113.3777614275639]] ... [[114.06464559061169]] [[112.96254669775898]] [[114.74266258672766]]]
Units,meter


<!-- ## Observed NPP -->


In [6]:
observed_npp = xr.open_zarr(path_to_npp)
observed_npp = observed_npp.sel(time=slice(TIME_START, TIME_END))
observed_npp = observed_npp.dropna("time", how="all")
observed_npp = observed_npp.resample(time="D").interpolate("linear")
observed_npp.load()

<xarray.Dataset> Size: 26kB
Dimensions:          (time: 1065, latitude: 1, longitude: 1)
Coordinates:
  * latitude         (latitude) float64 8B 22.75
  * longitude        (longitude) int64 8B -158
  * time             (time) datetime64[ns] 9kB 2005-01-31 ... 2007-12-31
Data variables:
    l12              (time, latitude, longitude) float64 9kB 273.4 ... 259.7
    l12_lowess_0.05  (time, latitude, longitude) float64 9kB 274.7 ... 258.9
Attributes:
    desc:     L12 multiplied by ZEU (from CMEMS).
    type:     climatology

### Observations


In [7]:
observations = xr.open_zarr(path_to_obs).load()
observations = observations.sel(latitude=LATITUDE, longitude=LONGITUDE, method="nearest")
observations = observations.resample(time="1D").mean().dropna("time")
observations = observations.pint.quantify().pint.to("mg/m^3")
observations = observations * epi_layer_depth
observations = observations.drop_vars("depth")
observations

Magnitude,[[[[157.92219924943342]]] [[[179.50290001191829]]] [[[206.9568833881889]]] [[[265.77390000839995]]] [[[281.78101853068074]]] [[[282.22919058899055]]] [[[295.88177275463875]]] [[[313.19485204421545]]] [[[314.33474301409973]]] [[[252.30876534149797]]] [[[204.84900876732033]]] [[[189.26359852904628]]] [[[157.92219924943342]]] [[[179.50290001191829]]] [[[206.9568833881889]]] [[[265.77390000839995]]] [[[281.78101853068074]]] [[[282.22919058899055]]] [[[295.88177275463875]]] [[[313.19485204421545]]] [[[314.33474301409973]]] [[[252.30876534149797]]] [[[204.84900876732033]]] [[[189.26359852904628]]] [[[157.92219924943342]]] [[[179.50290001191829]]] [[[206.9568833881889]]] [[[265.77390000839995]]] [[[281.78101853068074]]] [[[282.22919058899055]]] [[[295.88177275463875]]] [[[313.19485204421545]]] [[[314.33474301409973]]] [[[252.30876534149797]]] [[[204.84900876732033]]] [[[189.26359852904628]]]]
Units,milligram/meter2
Magnitude,[[[[237.0516536022709]]] [[[301.6376319958857]]] [[[337.83226845879244]]] [[[414.48959622350367]]] [[[389.1909678368789]]] [[[421.89365674709336]]] [[[424.06609922351373]]] [[[426.77207432265106]]] [[[418.1004648101298]]] [[[374.8468152741032]]] [[[333.9713931237319]]] [[[290.2018113694168]]] [[[237.0516536022709]]] [[[301.6376319958857]]] [[[337.83226845879244]]] [[[414.48959622350367]]] [[[389.1909678368789]]] [[[421.89365674709336]]] [[[424.06609922351373]]] [[[426.77207432265106]]] [[[418.1004648101298]]] [[[374.8468152741032]]] [[[333.9713931237319]]] [[[290.2018113694168]]] [[[237.0516536022709]]] [[[301.6376319958857]]] [[[337.83226845879244]]] [[[414.48959622350367]]] [[[389.1909678368789]]] [[[421.89365674709336]]] [[[424.06609922351373]]] [[[426.77207432265106]]] [[[418.1004648101298]]] [[[374.8468152741032]]] [[[333.9713931237319]]] [[[290.2018113694168]]]]
Units,milligram/meter2


observations


Select the kind of observation you want to use.


In [8]:
observations = observations.isel(time=slice(STABILIZATION_TIME, None))
observations

Magnitude,[[[[157.92219924943342]]] [[[179.50290001191829]]] [[[206.9568833881889]]] [[[265.77390000839995]]] [[[281.78101853068074]]] [[[282.22919058899055]]] [[[295.88177275463875]]] [[[313.19485204421545]]] [[[314.33474301409973]]] [[[252.30876534149797]]] [[[204.84900876732033]]] [[[189.26359852904628]]] [[[157.92219924943342]]] [[[179.50290001191829]]] [[[206.9568833881889]]] [[[265.77390000839995]]] [[[281.78101853068074]]] [[[282.22919058899055]]] [[[295.88177275463875]]] [[[313.19485204421545]]] [[[314.33474301409973]]] [[[252.30876534149797]]] [[[204.84900876732033]]] [[[189.26359852904628]]]]
Units,milligram/meter2
Magnitude,[[[[237.0516536022709]]] [[[301.6376319958857]]] [[[337.83226845879244]]] [[[414.48959622350367]]] [[[389.1909678368789]]] [[[421.89365674709336]]] [[[424.06609922351373]]] [[[426.77207432265106]]] [[[418.1004648101298]]] [[[374.8468152741032]]] [[[333.9713931237319]]] [[[290.2018113694168]]] [[[237.0516536022709]]] [[[301.6376319958857]]] [[[337.83226845879244]]] [[[414.48959622350367]]] [[[389.1909678368789]]] [[[421.89365674709336]]] [[[424.06609922351373]]] [[[426.77207432265106]]] [[[418.1004648101298]]] [[[374.8468152741032]]] [[[333.9713931237319]]] [[[290.2018113694168]]]]
Units,milligram/meter2


In [9]:
# observations_selected = observations[["day_lowess_0.2", "night_lowess_0.2"]].rename(
#     {"day_lowess_0.2": "day", "night_lowess_0.2": "night"}
# )

obs_night = observations["night"]
obs_day = observations["day"]

Remove the X first months to let the model reach the stationary state.


Create structure for SeapoPym simulation.


In [10]:
forcing_parameters = ForcingParameter(
    temperature=ForcingUnit(forcing=forcing["T"]), primary_production=ForcingUnit(forcing=observed_npp["l12"])
)

2025-06-11 11:42:56,307 :: Seapodym ::  WARNING ::
|	l12 unit is milligram / day / meter ** 2, it will be converted to kilogram / day / meter ** 2.

2025-06-11 11:42:56,308 :: Seapodym ::  WARNING ::
|	l12 unit is milligram / day / meter ** 2, it will be converted to kilogram / day / meter ** 2.



## Setup the parameters and the cost function


In [ ]:
functional_groups = [
    NoTransportFunctionalGroup(
        name="D1N1",
        day_layer=0,
        night_layer=0,
        energy_transfert=Parameter("D1N1_energy_transfert", 0.0001, 0.5),
        gamma_tr=Parameter("D1N1_gamma_tr", -0.5, -0.0001),
        tr_0=Parameter("D1N1_tr_0", 0, 50),
        gamma_lambda_temperature=Parameter("D1N1_gamma_lambda_temperature", 0, 0.5),
        lambda_temperature_0=Parameter("D1N1_lambda_temperature_0", 1 / 500, 1),
    ),
    NoTransportFunctionalGroup(
        name="D2N1",
        day_layer=1,
        night_layer=0,
        energy_transfert=Parameter("D2N1_energy_transfert", 0.0001, 0.5),
        gamma_tr=Parameter("D2N1_gamma_tr", -0.5, -0.0001),
        tr_0=Parameter("D2N1_tr_0", 0, 50),
        gamma_lambda_temperature=Parameter("D2N1_gamma_lambda_temperature", 0, 0.5),
        lambda_temperature_0=Parameter("D2N1_lambda_temperature_0", 1 / 500, 1),
    ),
]

In [12]:
model_generator = NoTransportModelGenerator(forcing_parameters=forcing_parameters)

In [13]:
cost_function = SimpleRootMeanSquareErrorCostFunction(
    model_generator=model_generator,
    observations=[
        TimeSeriesObservation(
            name="Hot climato day",
            observation=obs_day,
            observation_type=DayCycle.DAY,
            observation_interval="1MS",
        ),
        TimeSeriesObservation(
            name="Hot climato night",
            observation=obs_night,
            observation_type=DayCycle.NIGHT,
            observation_interval="1MS",
        ),
    ],
    functional_groups=functional_groups,
    root_mse=True,
    normalized_mse=True,
    centered_mse=False,
)

Set the genetic algorithm meta parameters.


In [14]:
genetic_algo_parameters = SimpleGeneticAlgorithmParameters(
    MUTPB=0.30,
    INDPB=0.2,
    ETA=5,
    CXPB=0.7,
    NGEN=5,
    POP_SIZE=500,
    cost_function_weight=(-1, -1),
)

Import or create the parameters to be optimized.


In [15]:
from seapopym_optimization.genetic_algorithm.simple_logbook import generate_logbook_with_sobol_sampling


logbook_path = Path(f"./{export_file_name}_logbook.parquet")
if logbook_path.exists():
    logbook = pd.read_parquet(logbook_path)
else:
    logbook = generate_logbook_with_sobol_sampling(
        functional_group_parameters=functional_groups, sample_number=256, fitness_name=["Hot climato"]
    )
logbook

category                                                      Parametre  \
name                                              D1N1_energy_transfert   
Generation Is_From_Previous_Generation Individual                         
0          False                       0                       0.464522   
                                       1                       0.262625   
                                       2                       0.464522   
                                       3                       0.464522   
                                       4                       0.464522   
...                                                                 ...   
                                       3067                    0.461432   
                                       3068                    0.461432   
                                       3069                    0.461432   
                                       3070                    0.461432   
                                       3071                    0.321242   

category                                                                     \
name                                              D1N1_lambda_temperature_0   
Generation Is_From_Previous_Generation Individual                             
0          False                       0                           0.458520   
                                       1                           0.458520   
                                       2                           0.503978   
                                       3                           0.458520   
                                       4                           0.458520   
...                                                                     ...   
                                       3067                        0.572998   
                                       3068                        0.572998   
                                       3069                        0.572998   
                                       3070                        0.572998   
                                       3071                        0.744901   

category                                                                         \
name                                              D1N1_gamma_lambda_temperature   
Generation Is_From_Previous_Generation Individual                                 
0          False                       0                               0.441978   
                                       1                               0.441978   
                                       2                               0.441978   
                                       3                               0.236121   
                                       4                               0.441978   
...                                                                         ...   
                                       3067                            0.220145   
                                       3068                            0.220145   
                                       3069                            0.220145   
                                       3070                            0.220145   
                                       3071                            0.150302   

category                                                                    \
name                                               D1N1_tr_0 D1N1_gamma_tr   
Generation Is_From_Previous_Generation Individual                            
0          False                       0           34.167544     -0.450900   
                                       1           34.167544     -0.450900   
                                       2           34.167544     -0.450900   
                                       3           34.167544     -0.450900   
                                       4            0.797080     -0.450900   
...                                                    

Finaly, create the Genetic Algorithm.


In [16]:
client = Client()
genetic_algo = SimpleGeneticAlgorithm(
    meta_parameter=genetic_algo_parameters,
    cost_function=cost_function,
    client=client,
    logbook=logbook,
    save=logbook_path,
)

/Users/adm-lehodey/Documents/Workspace/Projects/Seapopym-optimisation/.venv/lib/python3.13/site-packages/distributed/node.py:187: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 63223 instead
  warnings.warn(


And watch the magic on the Dask dashboard :


In [17]:
genetic_algo.client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:63223/status,
Dashboard: http://127.0.0.1:63223/status,Workers: 4
Total threads: 12,Total memory: 48.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:63224,Workers: 0
Dashboard: http://127.0.0.1:63223/status,Total threads: 0
Started: Just now,Total memory: 0 B
Comm: tcp://127.0.0.1:63238,Total threads: 3
Dashboard: http://127.0.0.1:63242/status,Memory: 12.00 GiB
Nanny: tcp://127.0.0.1:63227,


## Run the optimization


In [18]:
viewer = genetic_algo.optimize()

Some individuals in the logbook have no fitness values. Re-evaluating the population.


## Optimization statistics


In [19]:
viewer.hall_of_fame()

category                                                      Parametre  \
name                                              D1N1_energy_transfert   
Generation Is_From_Previous_Generation Individual                         
4          False                       93                      0.378239   
3          False                       91                      0.378239   
4          False                       80                      0.378239   
                                       251                     0.499739   
1          False                       139                     0.378239   
...                                                                 ...   
0          False                       392                     0.443741   
                                       393                     0.443741   
                                       394                     0.443741   
                                       391                     0.443741   
                                       1659                    0.177529   

category                                                                     \
name                                              D1N1_lambda_temperature_0   
Generation Is_From_Previous_Generation Individual                             
4          False                       93                          0.709242   
3          False                       91                          0.709242   
4          False                       80                          0.709242   
                                       251                         0.766008   
1          False                       139                         0.709242   
...                                                                     ...   
0          False                       392                         0.006096   
                                       393                         0.006096   
                                       394                         0.006096   
                                       391                         0.006096   
                                       1659                        0.000530   

category                                                                         \
name                                              D1N1_gamma_lambda_temperature   
Generation Is_From_Previous_Generation Individual                                 
4          False                       93                              0.003501   
3          False                       91                              0.003501   
4          False                       80                              0.005433   
                                       251                             0.023094   
1          False                       139                             0.005433   
...                                                                         ...   
0          False                       392                             0.008756   
                                       393                             0.008756   
                                       394                             0.008756   
                                       391                             0.008756   
                                       1659                            0.060604   

category                                                                    \
name                                               D1N1_tr_0 D1N1_gamma_tr   
Generation Is_From_Previous_Generation Individual                            
4          False                       93          22.470146     -0.364428   
3          False                       91          22.470146     -0.364428   
4          False                       80          22.470146     -0.310022   
                                       251         22.470146     -0.310022   
1          False                       139         22.470146     -0.310022   
...                                                    

In [20]:
viewer.fitness_evolution(points="all")

In [21]:
viewer.parameters_standardized_deviation()

In [22]:
viewer.shannon_entropy(bins=100_000)

In [23]:
fig = viewer.box_plot(3, nbest=500)
fig.show()

In [34]:
parameter_groups = [
    [
        "D1N1_energy_transfert",
        "D1N1_lambda_temperature_0",
        "D1N1_gamma_lambda_temperature",
        "D1N1_tr_0",
        "D1N1_gamma_tr",
    ],
    [
        "D2N1_energy_transfert",
        "D2N1_lambda_temperature_0",
        "D2N1_gamma_lambda_temperature",
        "D2N1_tr_0",
        "D2N1_gamma_tr",
    ],
]

In [35]:
fig = viewer.parallel_coordinates(
    nbest=10, reversescale=True, unselected_opacity=0.05, parameter_groups=parameter_groups, uniformed=False
)

for group in fig:
    display(group)